In [8]:
%load_ext autoreload
%autoreload 2

from src.cleaner.auto_cleaner import clean_text,ml_clean_text
import glob
import json
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
#クリーニングする処理
text="衝撃!!|中山トンネルは、上越新幹線高崎駅 - 上毛高原駅間にある総延長14,857 mの複線鉄道トンネルであり高崎方面から進行すると榛名トンネルの次、二番目に通過するトンネルである。建設中に2回の大出水事故を起こして難工事を極め、2回の経路変更によりようやく完成したが、経路変更のためにトンネル内に半径1,500 mの曲線ができてしまい、営業速度240 km/hの新幹線がトンネル内の曲線部分を通過するときには160 km/hに減速せざるをえなくなった。"
#text="中山トンネルは、上越新幹線高崎駅 - 上毛高原駅間にある総延長14,857 mの複線鉄道トンネルであり高崎方面から進行すると榛名トンネルの次、二番目に通過するトンネルである。建設中に2回の大出水事故を起こして難工事を極め、2回の経路変更によりようやく完成したが、経路変更のためにトンネル内に半径1,500 mの曲線ができてしまい、営業速度240 km/hの新幹線がトンネル内の曲線部分を通過するときには160 km/hに減速せざるをえなくなった。"
print(clean_text(text))

中山トンネルは、上越新幹線高崎駅-上毛高原駅間にある総延長14,857 mの複線鉄道トンネルであり高崎方面から進行すると榛名トンネルの次、二番目に通過するトンネルである。建設中に2回の大出水事故を起こして難工事を極め、2回の経路変更によりようやく完成したが、経路変更のためにトンネル内に半径1,500 mの曲線ができてしまい、営業速度240 km/hの新幹線がトンネル内の曲線部分を通過するときには160 km/hに減速せざるをえなくなった。


In [10]:
#以下､クラスタリング後のjsonlファイルを読み込む例｡
#datasetsライブラリから､適当にmc4-jaなどを読み込んで問題ナシ｡
path_list=glob.glob("../data/categorized/**/*.jsonl",recursive=True)


In [11]:
len(path_list)

5973715

In [12]:
random.shuffle(path_list)

In [13]:
line_list=[]
path=random.choice(path_list)
with open(path,"r") as f:
    #1行ずつ読み込む
    for line in f:
        text=json.loads(line)["text"]
        cleaned=clean_text(text)
        print(cleaned)
        break

汗臭い原因は汗の質にあった!運動で適度に汗をかこう
脇や汗の臭い対策!
特許申請中のノアンデは、速攻3秒で100%を除菌
汗がたくさん出る暑い季節は、わきがが気になる人も多いかもしれません。
しかし実は寒い季節でもわきがの臭いは無くなってはいないのです。
何故ならば冬でも人間は汗をかき、人によってはその汗が匂いの原因となるからです。
ところがスポーツ選手や激しいトレーニングをして毎日多くの汗をかく人が必ずしも汗臭いとは限りません。
逆に普段ほとんど運動しない人の方が少しだけ汗をかいただけで汗臭い嫌な匂いを発している印象があります。
実は同じ汗でもその質の違いで匂いも変わってくるのです。
嫌な汗臭い臭いを変える対策はあるのでしょうか。
上手に汗を出す為の対策
すぐに汗だくになってしまう人と、暑くても平気な人がいます。
しかし汗かきだからといって必ずしも汗臭い訳ではない事は誰もが思う事でしょう。
常に汗を拭いていても臭くない人も多いのです。
こうした人は汗がサラサラしており拭いてしまえばすぐに乾燥して匂いも残らない事が多いのです。
逆に普段は汗をかかなくても少量の汗で臭ってしまう人もいます。
汗がベタベタしており、拭いても汗の成分が皮膚に残ってしまい、それが細菌と合わさって腐敗臭を出すと言われています。
これは汗に塩分の他にミネラル成分が多い為だと考えられています。
ではどうしたらベタベタ汗でなくサラサラ汗になるのでしょうか。
お酒をたくさん飲んだ翌朝や、体調が悪い時には大量の寝汗をかくものですが、こうした寝汗は酷く嫌な匂いがする事は誰でも経験済みでしょう。
明らかにスポーツでの汗とは違っています。
暑い場合や運動した場合と違って不自然な汗は通常より余分な成分が含まれているからです。
しかし普段ほとんど運動をしない人は汗腺が上手に働かなくなっており、暑さによって少量の汗を出した場合でもこのようなベタベタ汗を出してしまう傾向があります。
特に普段はエアコンの効いた室内にいるような人は一層汗をかく機会が少ないので、ほんの少しの暑さでこの嫌な匂いがする汗が出てしまうのです。
人間は体の機能として汗をかくように出来ているので、暑い場合、運動をした場合には適度に汗をかくのが正常です。
正常なサラサラ汗であれば本来は嫌な匂いはしないはずなのです。
いつのまにか汗を出す事に慣れない体

In [14]:
text

'汗臭い原因は汗の質にあった!運動で適度に汗をかこう|わきがトーク-ワキのニオイ対策情報\n運動で適度に汗をかこう\n生活習慣\n汗臭い原因は汗の質にあった!運動で適度に汗をかこう\n脇や汗の臭い対策!\n女性・子供におすすめアイテム3選\nクリアネオで制汗・徹底殺菌!ニオイ&汗ジミ対策\nラポマインで保湿・殺菌制汗を1度でシンプルケア\n特許申請中のノアンデは、速攻3秒で100%を除菌\n汗がたくさん出る暑い季節は、わきがが気になる人も多いかもしれません。\nしかし実は寒い季節でもわきがの臭いは無くなってはいないのです。\n何故ならば冬でも人間は汗をかき、人によってはその汗が匂いの原因となるからです。\nところがスポーツ選手や激しいトレーニングをして毎日多くの汗をかく人が必ずしも汗臭いとは限りません。\n逆に普段ほとんど運動しない人の方が少しだけ汗をかいただけで汗臭い嫌な匂いを発している印象があります。\n実は同じ汗でもその質の違いで匂いも変わってくるのです。\n嫌な汗臭い臭いを変える対策はあるのでしょうか。\nサラサラ汗とベタベタ汗の違い\n汗をたくさん出すようにする\n上手に汗を出す為の対策\nサラサラ汗とベタベタ汗の違い\nすぐに汗だくになってしまう人と、暑くても平気な人がいます。\nしかし汗かきだからといって必ずしも汗臭い訳ではない事は誰もが思う事でしょう。\n常に汗を拭いていても臭くない人も多いのです。\nこうした人は汗がサラサラしており拭いてしまえばすぐに乾燥して匂いも残らない事が多いのです。\n逆に普段は汗をかかなくても少量の汗で臭ってしまう人もいます。\n汗がベタベタしており、拭いても汗の成分が皮膚に残ってしまい、それが細菌と合わさって腐敗臭を出すと言われています。\nこれは汗に塩分の他にミネラル成分が多い為だと考えられています。\nではどうしたらベタベタ汗でなくサラサラ汗になるのでしょうか。\n汗をたくさん出すようにする\nお酒をたくさん飲んだ翌朝や、体調が悪い時には大量の寝汗をかくものですが、こうした寝汗は酷く嫌な匂いがする事は誰でも経験済みでしょう。\n明らかにスポーツでの汗とは違っています。\n暑い場合や運動した場合と違って不自然な汗は通常より余分な成分が含まれているからです。\nしかし普段ほとんど運動をしない人は汗腺が上手に働かなくなって

In [15]:
print(clean_text(text))

汗臭い原因は汗の質にあった!運動で適度に汗をかこう
脇や汗の臭い対策!
特許申請中のノアンデは、速攻3秒で100%を除菌
汗がたくさん出る暑い季節は、わきがが気になる人も多いかもしれません。
しかし実は寒い季節でもわきがの臭いは無くなってはいないのです。
何故ならば冬でも人間は汗をかき、人によってはその汗が匂いの原因となるからです。
ところがスポーツ選手や激しいトレーニングをして毎日多くの汗をかく人が必ずしも汗臭いとは限りません。
逆に普段ほとんど運動しない人の方が少しだけ汗をかいただけで汗臭い嫌な匂いを発している印象があります。
実は同じ汗でもその質の違いで匂いも変わってくるのです。
嫌な汗臭い臭いを変える対策はあるのでしょうか。
上手に汗を出す為の対策
すぐに汗だくになってしまう人と、暑くても平気な人がいます。
しかし汗かきだからといって必ずしも汗臭い訳ではない事は誰もが思う事でしょう。
常に汗を拭いていても臭くない人も多いのです。
こうした人は汗がサラサラしており拭いてしまえばすぐに乾燥して匂いも残らない事が多いのです。
逆に普段は汗をかかなくても少量の汗で臭ってしまう人もいます。
汗がベタベタしており、拭いても汗の成分が皮膚に残ってしまい、それが細菌と合わさって腐敗臭を出すと言われています。
これは汗に塩分の他にミネラル成分が多い為だと考えられています。
ではどうしたらベタベタ汗でなくサラサラ汗になるのでしょうか。
お酒をたくさん飲んだ翌朝や、体調が悪い時には大量の寝汗をかくものですが、こうした寝汗は酷く嫌な匂いがする事は誰でも経験済みでしょう。
明らかにスポーツでの汗とは違っています。
暑い場合や運動した場合と違って不自然な汗は通常より余分な成分が含まれているからです。
しかし普段ほとんど運動をしない人は汗腺が上手に働かなくなっており、暑さによって少量の汗を出した場合でもこのようなベタベタ汗を出してしまう傾向があります。
特に普段はエアコンの効いた室内にいるような人は一層汗をかく機会が少ないので、ほんの少しの暑さでこの嫌な匂いがする汗が出てしまうのです。
人間は体の機能として汗をかくように出来ているので、暑い場合、運動をした場合には適度に汗をかくのが正常です。
正常なサラサラ汗であれば本来は嫌な匂いはしないはずなのです。
いつのまにか汗を出す事に慣れない体

In [16]:
print(text)

汗臭い原因は汗の質にあった!運動で適度に汗をかこう|わきがトーク-ワキのニオイ対策情報
運動で適度に汗をかこう
生活習慣
汗臭い原因は汗の質にあった!運動で適度に汗をかこう
脇や汗の臭い対策!
女性・子供におすすめアイテム3選
クリアネオで制汗・徹底殺菌!ニオイ&汗ジミ対策
ラポマインで保湿・殺菌制汗を1度でシンプルケア
特許申請中のノアンデは、速攻3秒で100%を除菌
汗がたくさん出る暑い季節は、わきがが気になる人も多いかもしれません。
しかし実は寒い季節でもわきがの臭いは無くなってはいないのです。
何故ならば冬でも人間は汗をかき、人によってはその汗が匂いの原因となるからです。
ところがスポーツ選手や激しいトレーニングをして毎日多くの汗をかく人が必ずしも汗臭いとは限りません。
逆に普段ほとんど運動しない人の方が少しだけ汗をかいただけで汗臭い嫌な匂いを発している印象があります。
実は同じ汗でもその質の違いで匂いも変わってくるのです。
嫌な汗臭い臭いを変える対策はあるのでしょうか。
サラサラ汗とベタベタ汗の違い
汗をたくさん出すようにする
上手に汗を出す為の対策
サラサラ汗とベタベタ汗の違い
すぐに汗だくになってしまう人と、暑くても平気な人がいます。
しかし汗かきだからといって必ずしも汗臭い訳ではない事は誰もが思う事でしょう。
常に汗を拭いていても臭くない人も多いのです。
こうした人は汗がサラサラしており拭いてしまえばすぐに乾燥して匂いも残らない事が多いのです。
逆に普段は汗をかかなくても少量の汗で臭ってしまう人もいます。
汗がベタベタしており、拭いても汗の成分が皮膚に残ってしまい、それが細菌と合わさって腐敗臭を出すと言われています。
これは汗に塩分の他にミネラル成分が多い為だと考えられています。
ではどうしたらベタベタ汗でなくサラサラ汗になるのでしょうか。
汗をたくさん出すようにする
お酒をたくさん飲んだ翌朝や、体調が悪い時には大量の寝汗をかくものですが、こうした寝汗は酷く嫌な匂いがする事は誰でも経験済みでしょう。
明らかにスポーツでの汗とは違っています。
暑い場合や運動した場合と違って不自然な汗は通常より余分な成分が含まれているからです。
しかし普段ほとんど運動をしない人は汗腺が上手に働かなくなっており、暑さによって少量の汗を出した場合でもこのようなベタベタ汗を出

In [17]:
from src.cleaner.TextClassifier import TextClassifier

In [18]:
classifier=TextClassifier()

In [19]:
classifier.train_fasttext(autotuneDuration=120,wordNgrams=2)

Outputting annotations
Annotations saved to annotations/text_labels
Training fasttext model


Progress: 100.0% Trials:  322 Best score:  0.846154 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  5527
Number of labels: 2
Progress: 100.0% words/sec/thread:   11775 lr:  0.000000 avg.loss:  0.647563 ETA:   0h 0m 0s


Model saved to annotations/text_labels/model.bin
Testing model
(66, 0.8333333333333334, 0.8333333333333334)


In [29]:
line_list=[]
path=random.choice(path_list)
with open(path,"r") as f:
    #1行ずつ読み込む
    for line in f:
        text=json.loads(line)["text"]
        cleaned=clean_text(text)
        if cleaned=="":
            continue

        #清掃前後で判定が変わるかをしらべる
        is_bad=classifier.predict(cleaned)
        is_bad_=classifier.predict(text)
        is_bad="noise" if is_bad==0 else "good"
        is_bad_="noise" if is_bad_==0 else "good"
        print(is_bad+"-"+(is_bad_)+": ",cleaned.replace("\n",""))

noise-noise:  で、初めての保険更新日に夫から「更新手続きやっといて。」と言われたわけですがなにもかもがチンプンカンプンだった私は生命保険会社に勤めている友人に「自動車保険やってもらえる?」と連絡するなど、今思うとあきれるような行動をしたりしました。
good-good:  このホームページは、各保険の概要についてご紹介したものです。取扱商品、各保険の名称や補償内容等は引受保険会社によって異なりますので、ご契約(団体契約の場合はご加入)にあたっては、必ず「重要事項説明書」をよくご確認ください。
good-good:  介護保険サービスの利用は、原則、1割又は2割分を利用者が負担し、残りを介護保険の保険給付で負担しています。ただし、交通事故等の第三者行為が原因で、要介護状態になった場合や要介護度が重度化して介護保険サービスを利用した場合は、その費用を加害者である第三者が負担する必要があります。書類が提出されたのち、町による第三者行為求償の要件等の確認後、第三者側(加害者・損害保険会社等)と町から委託された埼玉県国民健康保険団体連合会が損害賠償の交渉を行います。なお、事故と介護給付との因果関係等が確認できない場合、求償できない場合があります。
noise-noise:  今年の3月、北九州市の病院へ一人の女性が運び込まれた。彼女の身体は、バセドウ病、糖尿病、胃潰瘍を患い全身ぼろぼろの状態であった。彼女は治療の甲斐も無く3日後亡くなった。実は彼女は保険証をもっておらず、病院で治療を受けることができなかったのだ。介護保険実施に伴い、今年度から国民健康保険料の滞納者に対する保険証取り上げが義務化された。保険証を取上げられた被保険者たちはどのような状況におかれているのか。保険証取り上げにより、「国民皆保険制度」はどうなってしまうのか。危機に陥っている国民健康保険制度の最前線を取材した。
good-good:  タウンで謝礼、かんたんに申し込みができるのに、アドバイザーなど24部門www。保険というみんなの生命保険アドバイザー鳥取県は、みんなの生命保険アドバイザー鳥取県とは、契約後のファイナンシャルプランナー充実www。みんなの生命保険アドバイザー鳥取県⇒管理人の妻がみんなの相談比較を?、約2000人の過去の利用者に、いつでも簡単を変えることができます。保険は保険で2

In [30]:

text="中山トンネルは、上越新幹線高崎駅 - 上毛高原駅間にある総延長14,857 mの複線鉄道トンネルであり高崎方面から進行すると榛名トンネルの次、二番目に通過するトンネルである。建設中に2回の大出水事故を起こして難工事を極め、2回の経路変更によりようやく完成したが、経路変更のためにトンネル内に半径1,500 mの曲線ができてしまい、営業速度240 km/hの新幹線がトンネル内の曲線部分を通過するときには160 km/hに減速せざるをえなくなった。"
ml_clean_text(text)

''

In [33]:
classifier.clean(text)

''